In [9]:
from collections import defaultdict
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from urb_frame import *
from urb_sequence import *
from plotly.offline import *
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

Read frames from KITTI sequence 00 into a list of frames (frame).

In [2]:
%%time
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_2'
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/00/image_3'

frame = []
for frameid in range(10):
    print(frameid)
    filename = '%06d.png'%(frameid)
    left_frame = Frame(LEFTDIR + '/' + filename)
    left_frame.get_framepoints()
    frame.append(left_frame)
    left_frame.read_corresponding(RIGHTDIR)
    left_frame.compute_depth()

0
1
2
3
4
5
6
7
8
9
CPU times: user 24.6 s, sys: 144 ms, total: 24.8 s
Wall time: 12.2 s


We can also manually compute the Depth of the points and show these

Turn the list of frames into a sequence. The first frame is automatically taken as the KeyFrame and its FramePoints are rerieved, depth is estimated, only the most confident points are kept.

If a new frame is added and its pose can not be reliably determined, the previous frame is turned in and they are tracked in the other frames.

In [3]:
seq = create_sequence(frame)

In [4]:
kf = seq.keyframes[2]
kf.get_pose()

array([[  9.99894537e-01,  -2.24523238e-03,   1.43482815e-02,
         -4.63461489e-03],
       [  2.21789896e-03,   9.99995696e-01,   1.92062440e-03,
          1.02355547e-02],
       [ -1.43525320e-02,  -1.88859881e-03,   9.99895214e-01,
         -2.36522893e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])

In [6]:
kf.get_pose_wrt(seq.keyframes[0])

array([[  9.99494576e-01,  -1.65213557e-03,   3.17468624e-02,
         -9.00927604e-02],
       [  1.42382033e-03,   9.99972972e-01,   7.21300308e-03,
          1.13086799e-02],
       [ -3.17579212e-02,  -7.16415563e-03,   9.99469914e-01,
         -5.29998042e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])

In [15]:
def plot_frame(frame):
    coords = frame.get_framepoints_xyz()
    layout = Layout(
                    scene = dict(
                    xaxis = dict(
                        nticks=4, range = [-100,100], mirror='ticks'),
                    yaxis = dict(
                        nticks=4, range = [0,200],),),
                    width=900,
                    height=300,
                    margin=dict(
                    r=20, l=10,
                    b=10, t=10)
                  )
    scatter = Scatter(x=coords[:,0], y=coords[:,2], mode='markers', marker=dict(size=2))
    iplot(dict(data=[scatter], layout=layout))



In [16]:
plot_frame(frame[0]);

In [12]:
%%bash
cp -r . ../trajectory
